# 하위 클래스화를 통한 새로운 레이어 및 모델 만들기

## 설정

In [11]:
# tensorflow.org/guide/keras/custom_layers_and_models
import tensorflow as tf
from tensorflow import keras

## `Layer` 클래스: 상태(가중치)와 일부 계산의 조합

Keras의 주요 추상화 중 하나는 `Layer` 클래스입니다. 레이어는 상태(레이어의 "가중치")와 입력에서 출력으로의 변환("호출, 레이어의 정방향 패스")을 모두 캡슐화합니다.

다음은 밀집 레이어입니다. 상태는 변수 `w` 및 `b`입니다.

In [2]:
class Linear(keras.layers.Layer):
    def __init__(self, units=32, input_dim=32):
        super(Linear, self).__init__()
        w_init = tf.random_normal_initializer() # weight 난수로 초기화
        self.w = tf.Variable(
            initial_value=w_init(shape=(input_dim, units), dtype="float32"),
            trainable=True,
        ) # (32,32)
        b_init = tf.zeros_initializer()
        self.b = tf.Variable(
            initial_value=b_init(shape=(units,), dtype="float32"), trainable=True
        ) # (32,)
    
    # 추상메소드 반드시 구현!
    def call(self, inputs):
        return tf.matmul(inputs, self.w) + self.b

파이썬 함수와 매우 유사한 일부 텐서 입력에서 레이어를 호출하여 레이어를 사용합니다.

In [6]:
x = tf.ones((2, 2))
linear_layer = Linear(4, 2) # (은닉층 뉴런수, 특성수)
y = linear_layer(x)   #  (2,2)(2,4)+(4,) 
print(y.shape) # (2,4)
print(y)

print(linear_layer.w)
print(linear_layer.b)
print(linear_layer.weights) # w, b

(2, 4)
tf.Tensor(
[[ 0.05613131 -0.06372713 -0.00400402 -0.11926006]
 [ 0.05613131 -0.06372713 -0.00400402 -0.11926006]], shape=(2, 4), dtype=float32)
<tf.Variable 'Variable:0' shape=(2, 4) dtype=float32, numpy=
array([[ 0.03808481,  0.0054635 ,  0.03868578, -0.00767764],
       [ 0.01804651, -0.06919064, -0.0426898 , -0.11158242]],
      dtype=float32)>
<tf.Variable 'Variable:0' shape=(4,) dtype=float32, numpy=array([0., 0., 0., 0.], dtype=float32)>
[<tf.Variable 'Variable:0' shape=(2, 4) dtype=float32, numpy=
array([[ 0.03808481,  0.0054635 ,  0.03868578, -0.00767764],
       [ 0.01804651, -0.06919064, -0.0426898 , -0.11158242]],
      dtype=float32)>, <tf.Variable 'Variable:0' shape=(4,) dtype=float32, numpy=array([0., 0., 0., 0.], dtype=float32)>]


가중치 `w`와 `b`는 레이어 속성으로 설정될 때 레이어에 의해 자동으로 추적됩니다.

In [7]:
assert linear_layer.weights == [linear_layer.w, linear_layer.b]

레이어에 가중치를 추가하는 더 빠른 바로 가기에 액세스할 수도 있습니다. `add_weight()` 메서드 사용

In [ ]:
# 더 간결한 코드: add_weight() 사용
class Linear(keras.layers.Layer):
    def __init__(self, units=32, input_dim=32):
        super(Linear, self).__init__()
        self.w = self.add_weight(
            shape=(input_dim, units), initializer="random_normal", trainable=True
        )
        self.b = self.add_weight(shape=(units,), initializer="zeros", trainable=True)

    def call(self, inputs):
        return tf.matmul(inputs, self.w) + self.b


x = tf.ones((2, 2))
linear_layer = Linear(4, 2) 
y = linear_layer(x)   # (2,2)(2,4)+(4,) => (2,4)
print(y)
print(linear_layer.w)
print(linear_layer.b)

## 레이어는 훈련 불가능한 가중치를 가질 수 있습니다

훈련 가능한 가중치 외에도 훈련 불가능한 가중치를 레이어에 추가할 수 있습니다. 이러한 가중치는 레이어를 훈련할 때 역전파 동안 고려되지 않아야 합니다.

훈련 불가능한 가중치를 추가 및 사용하는 방법은 다음과 같습니다.

In [10]:
import numpy as np

total = tf.Variable(np.zeros((2,2)))    # 변수인 경우, 대입 가능
# total = tf.constant(np.zeros((2,2)))  # 상수인 경우, 대입 불가

total.assign_add(np.array([[1.,1.],
                           [0.,0.]]))

print(total)

<tf.Variable 'Variable:0' shape=(2, 2) dtype=float64, numpy=
array([[1., 1.],
       [0., 0.]])>


In [29]:
import numpy as np
total = tf.constant(np.zeros((2,2)))
# total.assign_add(np.array([[1.,1.],[0.,0.]]))
print(total)

tf.Tensor(
[[0. 0.]
 [0. 0.]], shape=(2, 2), dtype=float64)


In [18]:
class ComputeSum(keras.layers.Layer):
    def __init__(self, input_dim):
        super(ComputeSum, self).__init__()
        # trainable = False는 Variable를 상수화 시키는 것이 아니다
        self.total = tf.Variable(initial_value=tf.zeros((input_dim,)), trainable=False)
        print(self.total)

    def call(self, inputs):
        self.total.assign_add(tf.reduce_sum(inputs, axis=0))
        return self.total

x = tf.ones((2, 2))
print(x)
my_sum = ComputeSum(2)
y = my_sum(x)
print(y.numpy()) # [2. 2.]
y = my_sum(x)
print(y.numpy()) # [4. 4.]

tf.Tensor(
[[1. 1.]
 [1. 1.]], shape=(2, 2), dtype=float32)
<tf.Variable 'Variable:0' shape=(2,) dtype=float32, numpy=array([0., 0.], dtype=float32)>
[2. 2.]
[4. 4.]


`layer.weights`의 일부이지만, 훈련 불가능한 가중치로 분류됩니다.

In [19]:
print("weights:", len(my_sum.weights))
print(my_sum.weights)

# trainable=False 학습대상에 포함되지 않음 (예: 전이학습 시 잘 훈련된 파라미터 보존)
print("non-trainable weights:", len(my_sum.non_trainable_weights))
print(my_sum.non_trainable_weights)

# trainable=True 학습대상에 포함함
# # It's not included in the trainable weights:
print("trainable_weights:", my_sum.trainable_weights)

weights: 1
[<tf.Variable 'Variable:0' shape=(2,) dtype=float32, numpy=array([4., 4.], dtype=float32)>]
non-trainable weights: 0
[]
trainable_weights: [<tf.Variable 'Variable:0' shape=(2,) dtype=float32, numpy=array([4., 4.], dtype=float32)>]


## 모범 사례: 입력 형상이 알려질 때까지 가중치 생성 지연하기

위의 `Linear` 레이어는 `__init__()`에서 가중치 `w` 및 `b`의 형상을 계산하는 데 사용되는 `input_dim` 인수를 사용했습니다.

In [39]:
class Linear(keras.layers.Layer):
    def __init__(self, units=32, input_dim=32):
        super(Linear, self).__init__()
        self.w = self.add_weight(
            shape=(input_dim, units), initializer="random_normal", trainable=True
        )
        self.b = self.add_weight(shape=(units,), initializer="zeros", trainable=True)

    def call(self, inputs):
        return tf.matmul(inputs, self.w) + self.b


대부분의 경우, 입력의 크기를 미리 알지 못할 수 있으며, 레이어를 인스턴스화한 후 얼마 지나지 않아 해당 값을 알게 되면 가중치를 지연 생성하고자 합니다.

Keras API에서는 레이어의 `build(self, inputs_shape)` 메서드에서 레이어 가중치를 만드는 것이 좋습니다. 다음과 같습니다.

In [20]:
class Linear(keras.layers.Layer):
    def __init__(self, units=32):
        super(Linear, self).__init__()
        self.units = units
        print("Linear.__init__()")
    
    # 콜백 함수, 생성자 1번만 호출
    def build(self, input_shape):
        self.w = self.add_weight(
            shape=(input_shape[-1], self.units),
            initializer="random_normal",
            trainable=True,
        )
        self.b = self.add_weight(
            shape=(self.units,), initializer="random_normal", trainable=True
        )
        print("Linear.build()")

    def call(self, inputs):
        print("Linear.call()")
        return tf.matmul(inputs, self.w) + self.b


레이어의 `__call__()` 메서드는 처음 호출될 때 자동으로 빌드를 실행합니다. 지연되어 사용하기 쉬운 레이어입니다.

In [21]:
# At instantiation, we don't know on what inputs this is going to get called
# 생성자만 호출
linear_layer = Linear(32)

# The layer's weights are created dynamically the first time the layer is called
y = linear_layer(x)
y = linear_layer(x)

Linear.__init__()
Linear.build()
Linear.call()
Linear.call()


## 재귀적으로 구성 가능한 레이어

또 다른 인스턴스의 속성으로 Layer 인스턴스를 할당하면 외부 레이어가 내부 레이어의 가중치를 추적하기 시작합니다.

`__init__()` 메서드에서 이러한 하위 레이어를 만드는 것이 좋습니다(하위 레이어에는 일반적으로 빌드 메서드가 있으므로 외부 레이어가 만들어질 때 빌드됩니다).

In [25]:
# Let's assume we are reusing the Linear class
# with a `build` method that we defined above.

# 외부 레이어(MLPBlock)가 내부 레이어(Linear) 출력
class MLPBlock(keras.layers.Layer):
    def __init__(self):
        super(MLPBlock, self).__init__()
        self.linear_1 = Linear(32)
        self.linear_2 = Linear(32)
        self.linear_3 = Linear(1)

    def call(self, inputs):
        x = self.linear_1(inputs)
        x = tf.nn.relu(x)
        x = self.linear_2(x)
        x = tf.nn.relu(x)
        return self.linear_3(x)


mlp = MLPBlock()
y = mlp(tf.ones(shape=(3, 64)))  # The first call to the `mlp` will create the weights
y = mlp(tf.ones(shape=(3, 64))) # The second call does not call build().

print(mlp.linear_1.weights[0])
print(mlp.weights[0])

print("weights:", len(mlp.weights))
print("weights:", mlp.weights) # MLPBlock 내부 레이어의 weight를 본인 weight처럼 출력
print("trainable weights:", len(mlp.trainable_weights))

Linear.__init__()
Linear.__init__()
Linear.__init__()
Linear.build()
Linear.call()
Linear.build()
Linear.call()
Linear.build()
Linear.call()
Linear.call()
Linear.call()
Linear.call()
<tf.Variable 'mlp_block_3/linear_14/Variable:0' shape=(64, 32) dtype=float32, numpy=
array([[-0.01812489, -0.03061845,  0.06115043, ..., -0.06353461,
        -0.03648038,  0.04672812],
       [-0.08203552, -0.13435142, -0.0501137 , ..., -0.03883396,
         0.06245974,  0.01497473],
       [-0.03659232,  0.03808283,  0.05323389, ..., -0.02977696,
        -0.00370063, -0.02395253],
       ...,
       [ 0.05888325, -0.05440469,  0.01115742, ...,  0.00182555,
        -0.04574198,  0.03653708],
       [-0.0266219 , -0.01706137,  0.04301069, ..., -0.05182215,
         0.02849604,  0.03947062],
       [-0.05902935, -0.0200877 ,  0.0256488 , ..., -0.09283221,
         0.00245771,  0.01027432]], dtype=float32)>
<tf.Variable 'mlp_block_3/linear_14/Variable:0' shape=(64, 32) dtype=float32, numpy=
array([[-0.0181248

## `add_loss()` 메서드

레이어의 `call()` 메서드를 작성할 때는 훈련 루프를 작성할 때 나중에 사용하려는 손실 텐서를 만들 수 있습니다. `self.add_loss(value)`를 호출하면 됩니다.

In [27]:
# A layer that creates an activity regularization loss
class ActivityRegularizationLayer(keras.layers.Layer):
    def __init__(self, rate=1e-2):
        super(ActivityRegularizationLayer, self).__init__()
        self.rate = rate

    # pass through: inputs 넣어 inputs 리턴
    def call(self, inputs):
        self.add_loss(self.rate * tf.reduce_sum(inputs)) # 이전층 output의 값이 낮아지는 효과
        return inputs

이러한 손실(내부 레이어에서 생성된 손실 포함)은 `layer.losses`를 통해 검색할 수 있습니다. 이 속성은 모든 `__call__()`이 시작될 때 최상위 레이어로 재설정되므로 `layer.losses`에는 항상 마지막 정방향 패스에서 생성된 손실값이 포함됩니다.

In [30]:
# OuterLayer 내 ActivityRegularizationLayer
class OuterLayer(keras.layers.Layer):
    def __init__(self):
        super(OuterLayer, self).__init__()
        self.activity_reg = ActivityRegularizationLayer(1e-2)

    def call(self, inputs):
        return self.activity_reg(inputs)


layer = OuterLayer()
assert len(layer.losses) == 0  # No losses yet since the layer has never been called
print(layer.losses) # []

_ = layer(tf.zeros(1, 1))
assert len(layer.losses) == 1  # We created one loss value
print(layer.losses) # add_loss로 인한 출력

# `layer.losses` gets reset at the start of each __call__
_ = layer(tf.ones(1, 1))
assert len(layer.losses) == 1  # This is the loss created during the call above
print(layer.losses) # 0.01

[]
[<tf.Tensor: shape=(), dtype=float32, numpy=0.0>]
[<tf.Tensor: shape=(), dtype=float32, numpy=0.01>]


또한, `loss` 속성에는 내부 레이어의 가중치에 대해 생성된 정규화 손실도 포함됩니다.

In [31]:
class OuterLayerWithKernelRegularizer(keras.layers.Layer):
    def __init__(self):
        super(OuterLayerWithKernelRegularizer, self).__init__()
        self.dense = keras.layers.Dense(
            32, kernel_regularizer=tf.keras.regularizers.l2(1e-3)  # add_loss(0.001 * np.sum(self.dense.w**2))
        ) # L2 규제(너무 정답에 가까워지는 것을 규제) 내 add_loss 구현되어 있음

    def call(self, inputs):
        return self.dense(inputs)


layer = OuterLayerWithKernelRegularizer()
_ = layer(tf.zeros((1, 1)))

# This is `1e-3 * sum(layer.dense.kernel ** 2)`,
# created by the `kernel_regularizer` above.
# 아래 두 값이 동일하다.
print(layer.losses)
print(0.001*np.sum(layer.weights[0].numpy()**2))

[<tf.Tensor: shape=(), dtype=float32, numpy=0.0020135557>]
0.0020135555267333984


이러한 손실은 다음과 같이 훈련 루프를 작성할 때 고려됩니다.

```python
# Instantiate an optimizer.
optimizer = tf.keras.optimizers.SGD(learning_rate=1e-3)
loss_fn = keras.losses.SparseCategoricalCrossentropy(from_logits=True)

# Iterate over the batches of a dataset.
for x_batch_train, y_batch_train in train_dataset:
  with tf.GradientTape() as tape:
    logits = layer(x_batch_train)  # Logits for this minibatch
    # Loss value for this minibatch
    loss_value = loss_fn(y_batch_train, logits)
    # Add extra losses created during this forward pass:
    loss_value += sum(model.losses)

  grads = tape.gradient(loss_value, model.trainable_weights)
  optimizer.apply_gradients(zip(grads, model.trainable_weights))
```

훈련 루프 작성에 대한 자세한 가이드는 [처음부터 훈련 루프 작성하기 가이드](https://www.tensorflow.org/guide/keras/writing_a_training_loop_from_scratch/)를 참조하세요.

이러한 손실은 `fit()`에서도 완벽하게 작동합니다(손실이 있는 경우, 자동으로 합산되어 주 손실에 추가됨).

In [33]:
import numpy as np

inputs = keras.Input(shape=(3,))
outputs = ActivityRegularizationLayer()(inputs)
model = keras.Model(inputs, outputs)

# If there is a loss passed in `compile`, thee regularization
# losses get added to it
model.compile(optimizer="adam", loss="mse")
model.fit(np.random.random((2, 3)), np.random.random((2, 3)))

# It's also possible not to pass any loss in `compile`,
# since the model already has a loss to minimize, via the `add_loss`
# call during the forward pass!
model.compile(optimizer="adam")
model.fit(np.random.random((2, 3)), np.random.random((2, 3)))

1/1 [==============================] - 0s 58ms/step - loss: 0.0309


## `add_metric()` 메서드

`add_loss()`와 마찬가지로, 레이어에는 훈련 중 수량의 이동 평균을 추적하기 위한 `add_metric()` 메서드도 있습니다.

다음 "로지스틱 엔드포인트" 레이어를 고려합니다. 입력 예측 및 목표치로 사용하여 `add_loss()`를 통해 추적하는 손실을 계산하고 `add_metric()`을 통해 추적하는 정확도 스칼라를 계산합니다.

In [34]:
class LogisticEndpoint(keras.layers.Layer):
    def __init__(self, name=None):
        super(LogisticEndpoint, self).__init__(name=name)
        self.loss_fn = keras.losses.BinaryCrossentropy(from_logits=True)
        self.accuracy_fn = keras.metrics.BinaryAccuracy()

    def call(self, targets, logits, sample_weights=None):
        # Compute the training-time loss value and add it
        # to the layer using `self.add_loss()`.
        print(targets)
        print(logits)
        loss = self.loss_fn(targets, logits, sample_weights)
        print(loss)
        self.add_loss(loss)

        # Log accuracy as a metric and add it
        # to the layer using `self.add_metric()`.
        acc = self.accuracy_fn(targets, logits, sample_weights)
        self.add_metric(acc, name="accuracy")

        # Return the inference-time prediction tensor (for `.predict()`).
        return tf.nn.softmax(logits)


이러한 방식으로 추적되는 메트릭은 `layer.metrics`를 통해 액세스할 수 있습니다.

In [35]:
layer = LogisticEndpoint()

targets = tf.ones((2, 2)) # 정답
logits = tf.constant([[1,1], # 예측값
                      [1,0]],dtype='float32')
y = layer(targets, logits)

print("layer.metrics:", layer.metrics)
print("current accuracy value:", float(layer.metrics[0].result()))

print(layer.losses)

tf.Tensor(
[[1. 1.]
 [1. 1.]], shape=(2, 2), dtype=float32)
tf.Tensor(
[[1. 1.]
 [1. 0.]], shape=(2, 2), dtype=float32)
tf.Tensor(0.40823308, shape=(), dtype=float32)
layer.metrics: [<keras.metrics.metrics.BinaryAccuracy object at 0x000002B5885454F0>]
current accuracy value: 0.75
[<tf.Tensor: shape=(), dtype=float32, numpy=0.40823308>]


`add_loss()`와 마찬가지로, 이러한 메트릭은 `fit()`의해 추적됩니다.

In [76]:
inputs = keras.Input(shape=(3,), name="inputs")
targets = keras.Input(shape=(10,), name="targets")
logits = keras.layers.Dense(10)(inputs)
predictions = LogisticEndpoint(name="predictions")(logits, targets) # loss 有

model = keras.Model(inputs=[inputs, targets], outputs=predictions)
model.compile(optimizer="adam")

data = {
    "inputs": np.random.random((3, 3)),
    "targets": np.random.random((3, 10)),
}
model.fit(data)

Tensor("Placeholder:0", shape=(None, 10), dtype=float32)
Tensor("Placeholder_1:0", shape=(None, 10), dtype=float32)
Tensor("predictions/binary_crossentropy/weighted_loss/value:0", shape=(), dtype=float32)
Tensor("model_2/dense_4/BiasAdd:0", shape=(None, 10), dtype=float32)
Tensor("IteratorGetNext:1", shape=(None, 10), dtype=float32)
Tensor("model_2/predictions/binary_crossentropy/weighted_loss/value:0", shape=(), dtype=float32)
Tensor("model_2/dense_4/BiasAdd:0", shape=(None, 10), dtype=float32)
Tensor("IteratorGetNext:1", shape=(None, 10), dtype=float32)
Tensor("model_2/predictions/binary_crossentropy/weighted_loss/value:0", shape=(), dtype=float32)
1/1 [==============================] - 0s 176ms/step - loss: 1.1042 - binary_accuracy: 0.0000e+00


## 레이어에서 선택적으로 직렬화를 활성화할 수 있습니다

- Object Serialization
- [함수 모델](https://www.tensorflow.org/guide/keras/functional/)의 일부로 사용자 정의 레이어를 직렬화해야 하는 경우, 선택적으로 `get_config()` 메서드를 구현할 수 있습니다.

In [37]:
class Linear(keras.layers.Layer):
    def __init__(self, units=32):
        super(Linear, self).__init__()
        self.units = units

    def build(self, input_shape):
        self.w = self.add_weight(
            shape=(input_shape[-1], self.units),
            initializer="random_normal",
            trainable=True,
        )
        self.b = self.add_weight(
            shape=(self.units,), initializer="random_normal", trainable=True
        )

    def call(self, inputs):
        return tf.matmul(inputs, self.w) + self.b

    def get_config(self):
        return {"units": self.units}


# Now you can recreate the layer from its config:
layer = Linear(64)
config = layer.get_config()
print(config)
new_layer = Linear.from_config(config)
print(new_layer.units)

{'units': 64}
64


기본 `Layer` 클래스의 `__init__()` 메서드는 일부 키워드 인수, 특히 `name` 및 `dtype`를 사용합니다. 이러한 인수를 `__init__()`의 부모 클래스에 전달하고 레이어 구성에 포함하는 것이 좋습니다.

In [39]:
def foo(a,b):
    print(a, b)
    
# foo((1,2)) # Error
foo(*(1,2)) # tuple unpacking - 호출 시 

1 2


In [43]:
def foo(*a): # *a: 가변인자
    sum = 0
    for num in a:
        sum += num
    print(sum)
    
# 인자가 몆개가 오든 OK
# foo(1,2,3)
# foo(1,2,3,4) 
foo(1,2,3,4,5)

15


In [44]:
def foo(a, b):
    print(a, b)

foo(b=1, a=2) # keyword parameter

2 1


In [46]:
def foo(**a): # **a: 가변형 keyword parameter
    print(a)

# 키워드 인자가 몆개가 오든 OK
# foo(b=1, a=2)
foo(b=1, a=2, c=3)

{'b': 1, 'a': 2, 'c': 3}


In [47]:
# 객체 직렬화
class Linear(keras.layers.Layer):
    def __init__(self, units=32, **kwargs):
        super(Linear, self).__init__(**kwargs)
        self.units = units

    def build(self, input_shape):
        self.w = self.add_weight(
            shape=(input_shape[-1], self.units),
            initializer="random_normal",
            trainable=True,
        )
        self.b = self.add_weight(
            shape=(self.units,), initializer="random_normal", trainable=True
        )

    def call(self, inputs):
        return tf.matmul(inputs, self.w) + self.b

    def get_config(self):
        config = super(Linear, self).get_config() # 부모의 속성
        print(config)
        config.update({"units": self.units}) # 본인의 속성
        return config


layer = Linear(64)
config = layer.get_config() # 사전형으로 받아서 keyword parameter(**kwargs)로 넘긴다
print(config)
new_layer = Linear.from_config(config)
config = layer.get_config()
print(config)

{'name': 'linear_21', 'trainable': True, 'dtype': 'float32'}
{'name': 'linear_21', 'trainable': True, 'dtype': 'float32', 'units': 64}
{'name': 'linear_21', 'trainable': True, 'dtype': 'float32'}
{'name': 'linear_21', 'trainable': True, 'dtype': 'float32', 'units': 64}


구성에서 레이어를 역직렬화할 때 유연성이 더 필요한 경우, `from_config()` 클래스 메서드를 재정의할 수도 있습니다. 다음은 `from_config()`의 기본 구현입니다.

```python
def from_config(cls, config):
  return cls(**config) # class reference / class를 객체로 만듬
```

직렬화 및 저장에 대한 자세한 내용은 [모델 저장 및 직렬화 가이드](https://www.tensorflow.org/guide/keras/save_and_serialize/)를 참조하세요.

In [4]:
def foo(**args): # 정의 시 **: 가변 키워드 파라메터
    print(args)
    
c = {'b': 2, 'a': 1}
# foo(b=2,a=1) # keyword parameter
foo(**c)  # 호출 시 **: dictionary unpacking

{'b': 2, 'a': 1}


In [9]:
# 키워드 가변인자, dictionary unpacking
class AAA():
    def __init__(self, **kwargs):
        print(kwargs)
        self.a = kwargs['a']
        self.b = kwargs['b']
    
    def get_config(self):
        return {"a": self.a, "b": self.b}
                
config = {'a': 1, 'b': 1}
aaa = AAA(**config)
# aaa.get_config()
bbb = AAA(**aaa.get_config()) # 또다른 객체 생성

{'a': 1, 'b': 1}
{'a': 1, 'b': 1}


## `call()` 메서드의 권한 있는 `training` 인수

일부 레이어, 특히 `BatchNormalization` 레이어와 `Dropout` 레이어는 훈련 및 추론 중에 서로 다른 동작을 갖습니다. 이러한 레이어의 경우, `call()` 메서드에서 `training`(boolean) 인수를 노출하는 것이 표준 관행입니다.

이 인수를 `call()`에서 노출하면 내장 훈련 및 평가 루프(예: `fit()`)를 사용하여 훈련 및 추론에서 레이어를 올바르게 사용할 수 있습니다.

In [12]:
class CustomDropout(keras.layers.Layer):
    def __init__(self, rate, **kwargs):
        super(CustomDropout, self).__init__(**kwargs)
        self.rate = rate

    # 훈련 시에만 작동, 추론 시에는 작동 안함
    def call(self, inputs, training=None):
        if training:
            return tf.nn.dropout(inputs, rate=self.rate)
        return inputs


## `call()` 메서드의 권한 있는 `mask` 인수

`call()`에서 지원되는 다른 권한 있는 인수는 `mask` 인수입니다.

이 인수는 모든 Keras RNN 레이어에서 볼 수 있습니다. 마스크는 시계열 데이터를 처리할 때 특정 입력 타임스텝을 건너뛰는 데 사용되는 부울 텐서(입력의 타임스텝당 하나의 부울 값)입니다.

Keras는 이전 레이어에서 마스크가 생성될 때 이를 지원하는 레이어에 대해 올바른 `mask` 인수를 `__call__()`에 자동으로 전달합니다. 마스크 생성 레이어는 `mask_zero=True` 레이어와 `Masking` 레이어로 구성된  `Embedding`입니다.

마스킹 및 마스킹 지원 레이어를 작성하는 방법에 대한 자세한 내용은 ["패딩 및 마스킹 이해하기"](https://www.tensorflow.org/guide/keras/masking_and_padding/) 가이드를 확인하세요.

## `Model` 클래스

일반적으로, `Layer` 클래스를 사용하여 내부 계산 블록을 정의하고 `Model` 클래스를 사용하여 훈련할 객체인 외부 모델을 정의합니다.

예를 들어, ResNet50 모델에는 `Layer`를 하위 클래스화하는 여러 ResNet 블록과 전체 ResNet50 네트워크를 포괄하는 단일 `Model`이 있습니다.

`Model` 클래스는 `Layer`와 같은 API를 가지며, 다음과 같은 차이점이 있습니다.

- 내장 훈련, 평가 및 예측 루프( `model.fit()` , `model.evaluate()`, `model.predict()`)를 제공합니다.
- `model.layers` 속성을 통해 내부 레이어의 목록을 노출합니다.
- 저장 및 직렬화 API(`save()`, `save_weights()`...)를 노출합니다.

효과적으로, `Layer` 클래스는 문서에서 일컫는 "레이어"("컨볼루션 레이어" 또는 "되풀이 레이어"에서와 같이) 또는 "블록"("ResNet 블록" 또는 "Inception 블록"에서와 같이)에 해당합니다.

한편, `Model` 클래스는 문서에서 "모델"("딥 러닝 모델"에서) 또는 "네트워크"( "딥 신경망"에서)로 지칭되는 것에 해당합다.

"`Layer` 클래스를 사용해야 할까요? 아니면 `Model` 클래스를 사용해야 할까요?"라는 질문이 있다면 자문해 보세요. `fit()`을 호출해야 할까? `save()`를 호출해야 할까? 만약 그렇다면 `Model`를 사용하세요. 그렇지 않다면(클래스가 더 큰 시스템의 블록이거나 직접 훈련을 작성하고 코드를 저장하기 때문에) `Layer`를 사용하세요.

예를 들어, 위의 mini-resnet 예제를 사용하여 `fit()`으로 훈련하고 `save_weights()`로 저장할 수 있는 `Model`을 빌드할 수 있습니다.

```python
class ResNet(tf.keras.Model):

    def __init__(self):
        super(ResNet, self).__init__()
        self.block_1 = ResNetBlock()
        self.block_2 = ResNetBlock()
        self.global_pool = layers.GlobalAveragePooling2D()
        self.classifier = Dense(num_classes)

    def call(self, inputs):
        x = self.block_1(inputs)
        x = self.block_2(x)
        x = self.global_pool(x)
        return self.classifier(x)


resnet = ResNet()
dataset = ...
resnet.fit(dataset, epochs=10)
resnet.save(filepath)
```

## 종합: 엔드 투 엔드 예제

지금까지 배운 내용은 다음과 같습니다.

- `Layer`는 상태(`__init__()` 또는 `build()`) 및 일부 계산(`call()`에서 정의)을 캡슐화합니다.
- 레이어를 재귀적으로 중첩하여 새롭고 더 큰 계산 블록을 만들 수 있습니다.
- 레이어는 `add_loss()` 및 `add_metric()`을 통해 메트릭뿐만 아니라 손실(일반적으로, 정규화 손실)을 생성 및 추적할 수 있습니다.
- 훈련하려는 외부 컨테이너는 `Model`입니다. `Model`은 `Layer`와 비슷하지만, 훈련 및 직렬화 유틸리티가 추가되었습니다.

이 모든 것을 엔드 투 엔드 예제에 넣어봅시다. VAE(Variational AutoEncoder)를 구현할 것이며, MNIST 숫자로 훈련할 것입니다.

VAE는 `Model`의 서브 클래스가 될 것이며 `Layer`를 하위 클래스화하는 중첩된 레이어 구성으로 빌드됩니다. 정규화 손실(KL 확산)을 제공합니다.

- VAE 참고: tensorflow.org/tutorials/generative/변이 자동 인코더

In [13]:
from tensorflow.keras import layers


class Sampling(layers.Layer):
    """Uses (z_mean, z_log_var) to sample z, the vector encoding a digit."""

    def call(self, inputs):
        z_mean, z_log_var = inputs
        batch = tf.shape(z_mean)[0]
        dim = tf.shape(z_mean)[1]
        epsilon = tf.keras.backend.random_normal(shape=(batch, dim))
        return z_mean + tf.exp(0.5 * z_log_var) * epsilon


class Encoder(layers.Layer):
    """Maps MNIST digits to a triplet (z_mean, z_log_var, z)."""

    def __init__(self, latent_dim=32, intermediate_dim=64, name="encoder", **kwargs):
        super(Encoder, self).__init__(name=name, **kwargs)
        self.dense_proj = layers.Dense(intermediate_dim, activation="relu")
        self.dense_mean = layers.Dense(latent_dim)
        self.dense_log_var = layers.Dense(latent_dim)
        self.sampling = Sampling()

    def call(self, inputs):
        x = self.dense_proj(inputs)
        z_mean = self.dense_mean(x)
        z_log_var = self.dense_log_var(x)
        z = self.sampling((z_mean, z_log_var))
        return z_mean, z_log_var, z


class Decoder(layers.Layer):
    """Converts z, the encoded digit vector, back into a readable digit."""

    def __init__(self, original_dim, intermediate_dim=64, name="decoder", **kwargs):
        super(Decoder, self).__init__(name=name, **kwargs)
        self.dense_proj = layers.Dense(intermediate_dim, activation="relu")
        self.dense_output = layers.Dense(original_dim, activation="sigmoid")

    def call(self, inputs):
        x = self.dense_proj(inputs)
        return self.dense_output(x)


class VariationalAutoEncoder(keras.Model):
    """Combines the encoder and decoder into an end-to-end model for training."""

    def __init__(
        self,
        original_dim,
        intermediate_dim=64,
        latent_dim=32,
        name="autoencoder",
        **kwargs
    ):
        super(VariationalAutoEncoder, self).__init__(name=name, **kwargs)
        self.original_dim = original_dim
        self.encoder = Encoder(latent_dim=latent_dim, intermediate_dim=intermediate_dim)
        self.decoder = Decoder(original_dim, intermediate_dim=intermediate_dim)

    def call(self, inputs):
        z_mean, z_log_var, z = self.encoder(inputs)
        reconstructed = self.decoder(z)
        # Add KL divergence regularization loss.
        kl_loss = -0.5 * tf.reduce_mean(
            z_log_var - tf.square(z_mean) - tf.exp(z_log_var) + 1
        )
        self.add_loss(kl_loss)
        return reconstructed # 변이


MNIST에 간단한 훈련 루프를 작성해 봅시다.

In [14]:
# 1) 직접 구현
original_dim = 784
vae = VariationalAutoEncoder(original_dim, 64, 32)

optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)
mse_loss_fn = tf.keras.losses.MeanSquaredError()

loss_metric = tf.keras.metrics.Mean()

(x_train, _), _ = tf.keras.datasets.mnist.load_data()
x_train = x_train.reshape(60000, 784).astype("float32") / 255

train_dataset = tf.data.Dataset.from_tensor_slices(x_train)
train_dataset = train_dataset.shuffle(buffer_size=1024).batch(64)

epochs = 2

# Iterate over epochs.
for epoch in range(epochs):
    print("Start of epoch %d" % (epoch,))

    # Iterate over the batches of the dataset.
    for step, x_batch_train in enumerate(train_dataset):
        with tf.GradientTape() as tape:
            reconstructed = vae(x_batch_train)
            # Compute reconstruction loss
            loss = mse_loss_fn(x_batch_train, reconstructed)
            loss += sum(vae.losses)  # Add KLD regularization loss

        grads = tape.gradient(loss, vae.trainable_weights)
        optimizer.apply_gradients(zip(grads, vae.trainable_weights))

        loss_metric(loss)

        if step % 100 == 0:
            print("step %d: mean loss = %.4f" % (step, loss_metric.result()))

Start of epoch 0
step 0: mean loss = 0.3485
step 100: mean loss = 0.1262
step 200: mean loss = 0.0995
step 300: mean loss = 0.0894
step 400: mean loss = 0.0844
step 500: mean loss = 0.0811
step 600: mean loss = 0.0789
step 700: mean loss = 0.0773
step 800: mean loss = 0.0761
step 900: mean loss = 0.0750
Start of epoch 1
step 0: mean loss = 0.0748
step 100: mean loss = 0.0741
step 200: mean loss = 0.0736
step 300: mean loss = 0.0731
step 400: mean loss = 0.0728
step 500: mean loss = 0.0724
step 600: mean loss = 0.0721
step 700: mean loss = 0.0718
step 800: mean loss = 0.0715
step 900: mean loss = 0.0713


VAE는 `Model`을 하위 클래스화하기 때문에 내장된 훈련 루프를 제공합니다. 따라서 다음과 같이 훈련할 수도 있습니다.

In [15]:
# 2) Keras 지원 함수 활용
vae = VariationalAutoEncoder(784, 64, 32)

optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)

vae.compile(optimizer, loss=tf.keras.losses.MeanSquaredError())
vae.fit(x_train, x_train, epochs=2, batch_size=64)

Epoch 1/2
938/938 [==============================] - 3s 2ms/step - loss: 0.0746
Epoch 2/2
938/938 [==============================] - 2s 2ms/step - loss: 0.0676


## 객체 지향 개발을 넘어: 함수형 API

이 예제가 너무 지나친 객체 지향 개발입니까? [함수형 API(Functional API)](https://www.tensorflow.org/guide/keras/functional/)를 사용하여 모델을 빌드할 수도 있습니다. 중요한 것은 하나의 스타일을 선택한다고 해서 다른 스타일로 작성된 구성 요소를 활용하지 못하는 것은 아닙니다. 항상 목적에 따라 다르게 선택할 수 있습니다.

예를 들어, 아래의 함수형 API 예제는 위 예제에서 정의한 것과 같은 `Sampling` 레이어를 재사용합니다.

In [16]:
original_dim = 784
intermediate_dim = 64
latent_dim = 32

# Define encoder model.
original_inputs = tf.keras.Input(shape=(original_dim,), name="encoder_input")
x = layers.Dense(intermediate_dim, activation="relu")(original_inputs)
z_mean = layers.Dense(latent_dim, name="z_mean")(x)
z_log_var = layers.Dense(latent_dim, name="z_log_var")(x)
z = Sampling()((z_mean, z_log_var))
encoder = tf.keras.Model(inputs=original_inputs, outputs=z, name="encoder")

# Define decoder model.
latent_inputs = tf.keras.Input(shape=(latent_dim,), name="z_sampling")
x = layers.Dense(intermediate_dim, activation="relu")(latent_inputs)
outputs = layers.Dense(original_dim, activation="sigmoid")(x)
decoder = tf.keras.Model(inputs=latent_inputs, outputs=outputs, name="decoder")

# Define VAE model.
outputs = decoder(z)
vae = tf.keras.Model(inputs=original_inputs, outputs=outputs, name="vae")

# Add KL divergence regularization loss.
kl_loss = -0.5 * tf.reduce_mean(z_log_var - tf.square(z_mean) - tf.exp(z_log_var) + 1)
vae.add_loss(kl_loss)

# Train.
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)
vae.compile(optimizer, loss=tf.keras.losses.MeanSquaredError())
vae.fit(x_train, x_train, epochs=3, batch_size=64)

Epoch 1/3
938/938 [==============================] - 3s 2ms/step - loss: 0.0749
Epoch 2/3
938/938 [==============================] - 2s 2ms/step - loss: 0.0676
Epoch 3/3
938/938 [==============================] - 2s 2ms/step - loss: 0.0675


자세한 정보는 [함수형 API 가이드](https://www.tensorflow.org/guide/keras/functional/)를 참고하세요.